<a href="https://colab.research.google.com/github/elifulton/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/Copy_of_LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [100]:
# the drill

import pandas as pd

# a little wget magic to retrieve the CSV zip file containing our data

!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-08-04 22:37:23--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 52.42.149.204, 52.42.71.195, 34.211.153.241, ...
Connecting to resources.lendingclub.com (resources.lendingclub.com)|52.42.149.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [           <=>      ]  22.26M  2.24MB/s    in 10s     

2020-08-04 22:37:34 (2.20 MB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [23339743]



In [101]:
# plug it, play it, burn it, rip it, drag it, drop it, zip, UNZIP it!

!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [113]:
# fixing for the header and the two last rows which are completely null

df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')

In [114]:
# fixing for the completely null columns as well

df = df.drop(['id', 'member_id', 'desc', 'url'], axis=1)

In [115]:
# function for converting percentages with leading space into usable floats

def int_rate_to_float(cell_contents):
  return float(cell_contents.strip().strip('%'))

In [116]:
# creating the cleaned_rate columns via both methods (.apply() and for loop), and even the cleaned_int_rate3 column completely comprised of 14.47

df['int_rate_cleaned'] = df['int_rate'].apply(int_rate_to_float)

clean_int_rates = []

for int_rate in df['int_rate']:
  clean_int_rates.append(int_rate_to_float(int_rate))

df['cleaned_int_rate2'] = pd.Series(clean_int_rates)

df['cleaned_int_rate3'] = int_rate_to_float(' 14.47%')

In [117]:
# creating the clean_emp_title function

def clean_emp_title(title):
  
  if isinstance(title, str):
    return title.strip().title()
  else:
    return "Unknown"

In [118]:
# creating new emp_title_cleaned column as well as overwriting the emp_title column

df['emp_title_cleaned'] = df['emp_title'].apply(clean_emp_title)
df['emp_title'] = df['emp_title'].apply(clean_emp_title)

In [119]:
# overwriting issue_d column to be in datetime format, then creating additional columns based off of new issue_d column

df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format=True)
df['credit_length_days'] = (df['issue_d'] - df['earliest_cr_line']).dt.days
df['credit_length_years'] = df['credit_length_days'] / 365

In [120]:
# new function to strip " months" from every term entry

def convert_term(term_length):
  return int(term_length.strip(" months"))

# now, we overwrite the original term column to only contain integers using our new function

df['term'] = df['term'].apply(convert_term)

In [121]:
# function to return the proper value for loan_status_is_great for any entry

def is_loan_status_great(status):
  if status == 'Current' or status == 'Fully Paid':
    return 1
  else:
    return 0

# creating new loan_status_is_great column based off of our new function

df['loan_status_is_great'] = df['loan_status'].apply(is_loan_status_great)

In [122]:
# replace NaNs in the last_pymnt_d column with placeholder ("Sep-1910")

df['last_pymnt_d'].fillna("Sep-1910", inplace=True)

# convert last_pymnt_d column to datetime entries only

df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)

# create last_pymnt_d_month and last_pymnt_d_year columns by retrieving the month and year

df['last_pymnt_d_month'] = df['last_pymnt_d'].dt.month
df['last_pymnt_d_year'] = df['last_pymnt_d'].dt.year

In [123]:
print(df.shape)
df.head()

(128412, 151)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned,cleaned_int_rate2,cleaned_int_rate3,emp_title_cleaned,issue_year,issue_month,credit_length_days,credit_length_years,loan_status_is_great,last_pymnt_d_month,last_pymnt_d_year
0,20000,20000,20000.0,36,14.47%,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,1979-06-01,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,2020-02-01,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,14.47,14.47,Bus Driver,2018,12,14428,39.528767,0,2,2020
1,12000,12000,12000.0,36,16.14%,422.72,C,C4,Sales Account Manager,4 years,RENT,70000.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,333xx,FL,18.96,0,2013-11-01,0,NaN,NaN,6,0,13663,61.8%,7,w,5931.69,5931.69,8324.660000,8324.66,6068.31,2256.35,0.0,0.0,0.0,2020-07-01,422.72,Aug-2020,Jul-2020,0,NaN,1,Individual,NaN,...,0,0,56264,34470,18500,34164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,CVD19SKIP,INCOMECURT,COMPLETE,2.0,0.00,Apr-2020,Jun-2020,Apr-2020,2.0,0.0,ACTIVE,173.05,6508.2,422.72,N,NaN,NaN,NaN,NaN,NaN,NaN,16.14,16.14,14.47,Sales Account Manager,2018,12,1856,5.084932,1,7,2020
2,7000,7000,6975.0,36,10.72%,228.25,B,B2,Business Analyst,< 1 year,MORTGAGE,97000.0,Source Verified,2018-12-01,Fully Paid,n,other,Other,201xx,VA,33.17,0,2011-09-01,1,NaN,NaN,18,0,1988,7.5%,30,f,0.00,0.00,7197.516809,7171.81,7000.00,197.52,0.0,0.0,0.0,2019-07-01,1346.14,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,0,0,677593,115121,21600,160050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72,10.72,14.47,Business Analyst,2018,12,2648,7.254795,1,7,2019
3,25000,25000,25000.0,60,16.14%,609.82,C,C4,Production Manager,5 years,MORTGAGE,45000.0,Not Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,703xx,LA,37.09,0,2003-09-01,0,NaN,NaN,7,0,8901,36.8%,21,w,0.00,0.00,26653.167580,26653.17,25000.00,1653.17,0.0,0.0,0.0,2019-04-01,24857.33,NaN,May-2019,0,NaN,1,Individual,NaN,...,0,0,73683,49524,18600,49483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,16.14,16.14,14.47,Production Manager,2018,12,5570,15.260274,1,4,2019
4,10400,10400,10400.0,36,12.98%,350.32,B,B5,Pro

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [ ]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [ ]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [ ]:
# %cd instacart_2017_05_01